In [3]:
import pandas as pd
import json 
import re
import math 
import ast 

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken
import time 

import os
from dotenv import load_dotenv
from openai import OpenAI

import pprint

In [4]:
# llama
client = AsyncOpenAI(
    base_url="https://api.scaleway.ai/f1ced67f-44e3-466f-952f-9f6ea4271e45/v1", # Scaleway's Generative APIs service URL
    api_key= os.getenv("SCW_SECRET_KEY") # Your unique API secret key from Scaleway
)
model="llama-3.1-70b-instruct"

In [5]:
def read_parquet(uploaded_file):
    return pd.read_parquet(uploaded_file)

df = read_parquet("//Users/anne/Desktop/Data4Good/ECF GenAI/4_channels_predictions_09_2023_09_2024.parquet")
df.reset_index(inplace=True)
df_exploded = df.explode('claims', ignore_index=True)
df_normalized = pd.json_normalize(df_exploded['claims'])
df_result = df_exploded.drop(columns=['claims']).join(df_normalized)
df_claims = df_result[(df_result['disinformation_score']=='high') & (df_result['pro_anti']!='pro-écologie')]
df_claims

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
67,f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c...,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
87,e57e39657004072e5f998d19e6a8a952ee8f35923b9072...,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
251,5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5c...,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
259,53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73775,f875741067fda8011d4aa9ad4c543e52227e31b525fed1...,2024-07-29 13:14:00,sur place voilà ce qu'ils restent par exemple ...,tf1,False,Information - Journal,JT 13h,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""feux"", ""timestamp"": 172225164506...",1,446,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative
74166,cdfed233896d49971897df448465dd43b79ae871b33adb...,2024-08-05 09:16:00,les insectes non on pense que c'est la chaleur...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""g\u00e9othermie"", ""timestamp"": 1...",2,479,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,

In [14]:
df_claims.shape

(1454, 21)

## Quotes in text

In [6]:
# les quote ne sont pas toujours dans les textes -> mesure de similarité entre claim et quote ? 
df_claims["quote_in_text"] = df_claims.apply(lambda row: row["quote"] in row["text"], axis=1)
df_claims.groupby('quote_in_text').count()['id']

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_10557/1830324615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims["quote_in_text"] = df_claims.apply(lambda row: row["quote"] in row["text"], axis=1)


quote_in_text
False    908
True     546
Name: id, dtype: int64

In [7]:
# quote not in text
not_exact = df_claims[df_claims.quote_in_text == False]

In [8]:
df_claims.loc[259]

id                         53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...
start                                                    2023-09-06 08:38:00
text                       je suis même allé plus loin je vais essayer de...
channel_name                                                         europe1
channel_is_radio                                                        True
channel_program_type                                  Information - Magazine
channel_program                                               Europe 1 Matin
themes                     ["changement_climatique_consequences", "ressou...
keywords                   [{"keyword": "\u00e9coresponsable", "timestamp...
num_keywords                                                               2
num_tokens                                                               614
analysis                   Cette allégation frôle la théorie du complot, ...
claim                      Le réchauffement climatique est causé par une ...

In [7]:
pprint.pprint(df_claims.loc[259].quote)
pprint.pprint(df_claims.loc[259].claim)
pprint.pprint(df_claims.loc[259].text)

('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('je suis même allé plus loin je vais essayer de me mettre à sa place vous '
 'laisser de vous mettre à la place de manuel bompard un moment dimitri il '
 'faut passer par là parce que si vous voulez faire avancer le vivre ensemble '
 'il faut essayer de comprendre les gens plutôt que de les juger je crois que '
 'ça fait pas du bien à la canicule mais pour répondre à votre question oui ce '
 "matin j'essaie de m'imaginer l'état d'esprit notre invité quand ils bien ici "
 "bon alors racontait alors il faut l'imaginée quand il est en approche de "
 "notre immeuble c'est-à-dire ben il est la marche sur les quais de seine avec "
 "son attaché de presse un moment il s'arrête il lève la tête yvonne la autres "
 "dent immeubles ira et là dedans eur

## Correct text

In [9]:
system_prompt = '''
    L'utilisateur fournira un extrait de 2 minutes d'une émission de télévision ou de radio accompagné d'une citation tirée de la transcription.

    **Tâche principale** :
    1. Corriger la transcription pour la rendre en français correct, en reformulant les phrases pour leur donner du sens si nécessaire, tout en respectant la phonétique.
    4. Supprimer les répétitions inutiles (ex. "je l'ai vu, je l'ai vu" devient "je l'ai vu").
    2. Transformer les nombres écrits en toutes lettres dans le texte en chiffres (ex. "quatre-vingt" devient 80).
    3. Diviser le texte en paragraphes si plusieurs sujets sont abordés.

    **Exception** :
    - Ne modifie pas la citation fournie par l'utilisateur, même si elle contient des erreurs. La citation (quote) doit rester identique dans le texte corrigé.

    **Exemple** :
    - Texte : elles doivent donc être renouvelés et sané col débuts en europe.  
    - Citation : elles doivent donc être renouvelés  
    - Correction : elles doivent donc être renouvelés, et ce n'est que le début en Europe.

    Retourner uniquement le texte corrigé, avec la citation inchangée.
'''

async def correct_text(
        text: str, quote: str, system_prompt: str=system_prompt, tpm_limit: int=2e6
    ):
    content = "Texte: {}, Citation: {}" .format(text, quote)
    if model=="gpt-4o-mini":
        encoding = tiktoken.encoding_for_model(model)
        prompt_tokens = len(encoding.encode(system_prompt))
        text_tokens = len(encoding.encode(content))
        total_tokens = prompt_tokens + text_tokens
        wait_time_s = 60 * total_tokens / tpm_limit
        await asyncio.sleep(wait_time_s)
    else:
        await asyncio.sleep(60 * 50 / tpm_limit)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_correction(texts: list[str], quotes: list[str]):
    semaphore = asyncio.Semaphore(10)
    
    async def bounded_detect_claim(text, quote):
        async with semaphore:
            return await correct_text(text, quote)
    
    correction = await tqdm.gather(*[bounded_detect_claim(text, quote) for text, quote in zip(texts, quotes)])
    return correction

In [15]:
start_time = time.time()
subset = df_claims.iloc[:10]
correction = await run_correction(subset.text, subset.quote)

elapsed_time = time.time() - start_time

print(f"Time taken for {len(subset)} items: {elapsed_time:.2f} seconds")

100%|██████████| 10/10 [00:23<00:00,  2.33s/it]

Time taken for 10 items: 23.47 seconds


In [ ]:
import pickle
import time

file_path = 'corrected.pickle'

start_time = time.time()
subset = df_claims.iloc[:500]
correction = await run_correction(subset.text, subset.quote)

elapsed_time = time.time() - start_time

print(f"Time taken for {len(subset)} items: {elapsed_time:.2f} seconds")

with open(file_path, 'wb') as file:
    pickle.dump(correction, file)

  0%|          | 0/500 [00:00<?, ?it/s]

In [16]:
correction

['Voici le texte corrigé :\n\nSérie que l\'on peut redire par fonction, juré déjà, quatre nominations aux Emmy Awards aux manettes, les producteurs de The Office, c\'est un phénomène aux États-Unis. Il s\'agit d\'un documentaire sur 12 jurés appelés pour un procès civil américain. Comment délibèrent-ils ? Quels sont leurs doutes, leurs responsabilités ? Dites-moi, un documentaire sur la justice américaine, ce n\'est pas tout à fait nouveau. Mais un petit détail fait toute la différence : ce procès est faux. Tous les jurés sont des acteurs, tous sauf un, Ronald Gladone. Il n\'est pas comédien, il ne sait pas qu\'il joue dans un faux documentaire dont il est en fait le héros.\n\nCe que j\'ai vu en salle d\'attente, il y a toutes les classes, tous les âges, toutes les origines, vraiment tout. Il y a tout le monde dans cette salle. Ronald, que vous entendez dans cet extrait, n\'est pas au bout de ses surprises. Mais à ce moment-là, ils ignorent totalement qu\'il est victime d\'une gigantes

In [11]:
pprint.pprint(df_claims.loc[87].text)

("salle d'attente il y a toutes les classes rageant tous les âges toutes les "
 'origines vraiment tout dire tout le monde dans cette salle vous avez une '
 "question euh dimitris c'est terrible c'est le caucus de l'histoire ces ronal "
 "que vous venez d'entendre il n'est pas au bout de ses surprises à ce "
 "moment-là ils ignorent totalement qu'il est victime d'une gigantesque "
 'caméras cachées qui a duré douze jours mêlant se moque de lui en regardant '
 "la série non justement c'est hilarant parce que les situations sont "
 'improbables déjà les jurés sont confinés pendant dix jours dans dix un jours '
 "hôtel dans ce un qui hôtel n'ce arrivent jamais qui dans n'la arrivent "
 'réalité jamais dans la réalité et on sait que le procès en question oppose '
 "la directrice d'une marque de vêtements à un ouvrier qui auraient uriné sur "
 "ces t-shirts à l'usine toutes les personnages sont sympas mais très bizarre "
 'il y a par exemple le giec transhumaniste qui trimballe un pantalon 

In [9]:
correction = await correct_text(df_claims.loc[87].text, df_claims.loc[87].claim)
pprint.pprint(correction)

("Salle d'attente, il y a toutes les classes, tous les âges, toutes les "
 'origines, vraiment tout. Dire que tout le monde est dans cette salle. Vous '
 "avez une question, euh Dimitris ? C'est terrible, c'est le cauchemar de "
 "l'histoire. Ce Ronald que vous venez d'entendre n'est pas au bout de ses "
 "surprises. À ce moment-là, ils ignorent totalement qu'il est victime d'une "
 'gigantesque caméra cachée qui a duré 12 jours. Mêlant se moque de lui en '
 "regardant la série, non, justement, c'est hilarant parce que les situations "
 'sont improbables. Déjà, les jurés sont confinés pendant 10 jours dans un '
 "hôtel, ce qui n'arrive jamais dans la réalité. Et on sait que le procès en "
 "question oppose la directrice d'une marque de vêtements à un ouvrier qui "
 "aurait uriné sur ces t-shirts à l'usine. Toutes les personnages sont sympas "
 'mais très bizarres. Il y a par exemple le GIEC, un transhumaniste qui '
 "trimballe un pantalon-chaise qui lui permet de s'asseoir quand il le 